In [4]:
import numpy as np
import pandas as pd

import iqplot

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

In [62]:
df = pd.read_csv('data/grant_complete.csv', comment='#')

df.head()

,band,beak depth (mm),beak length (mm),species,year
0,20123,8.05,9.25,fortis,1973
1,20126,10.45,11.35,fortis,1973
2,20128,9.55,10.15,fortis,1973
3,20129,8.75,9.95,fortis,1973
4,20133,10.15,11.55,fortis,1973


In [63]:
df = df.loc[
    (df['species'] == 'scandens') & (df['year'].isin([1975, 2012])),
    ['year', 'beak depth (mm)', 'beak length (mm)']
]
                                      
# John tukey fast fourier transform, and book about exploratory data analysis EDA
# First steps always explore the data before any kind of stats and analysis
# Be sure experiment is solid !
# Mention Gravitotional wave paper that also used Jupyter notebook
# https://arxiv.org/pdf/1602.03837.pdf

In [7]:
p = iqplot.ecdf(
    data=df,
    q='beak depth (mm)',
    cats='year',
)

bokeh.io.show(p)

In [8]:
# Did it really shifted to the right, narrower ? or effect sample size ?
# But they measure all subjects of population!
# Can still be stochasticity population, just by chance (cf. finite not infinite number of birds)

In [9]:
bd_1975 = df.loc[df['year'] == 1975, 'beak depth (mm)'].values
bd_2012 = df.loc[df['year'] == 2012, 'beak depth (mm)'].values

np.mean(bd_1975), np.mean(bd_2012)

(8.959999999999999, 9.188492063492063)

In [14]:
# Difference of mean over 200 microns
# Whales (= mammals) evolved for quite long time to reach that size, only microns for each generation.
# Always thinking scientific question we are looking at, great scale. Never loose sight you are talking about real living world.

# Real difference or due to sample size !
# Need to use confidence of interval.
# If I were doing the experiment again (if possible), what would I see?

# Discussion plugin principle. mean
# Do it for confidence interval
# For that, use bootstrapping method. Since use measurement we already have to save ourselves, get more info.
# Brad Efron in 1979, proved that bootstraping good method for confidence interval.

In [38]:
rg = np.random.default_rng()

In [39]:
bs_sample = rg.choice(bd_1975, replace=True, size=len(bd_1975))

In [40]:
bs_sample

array([ 8.6 , 10.4 ,  9.1 , 10.4 ,  8.9 ,  8.5 ,  9.04,  8.4 ,  8.9 ,
        9.1 ,  9.8 ,  9.44,  8.5 ,  8.8 ,  8.9 ,  9.2 ,  8.  ,  8.8 ,
        8.1 ,  9.44,  9.1 , 10.4 ,  9.1 ,  8.9 ,  9.1 ,  9.  ,  8.6 ,
        9.2 ,  8.  , 10.4 ,  9.1 ,  8.5 ,  9.2 ,  9.04,  9.8 ,  8.5 ,
        8.3 ,  9.1 ,  8.6 ,  9.  ,  9.1 ,  9.  ,  8.3 ,  9.8 ,  9.8 ,
        8.2 ,  9.9 ,  9.45,  8.8 ,  8.6 ,  8.  ,  8.8 , 10.1 ,  9.7 ,
        9.  ,  9.6 ,  8.7 ,  8.3 ,  8.1 ,  9.2 ,  9.05,  8.7 ,  9.1 ,
        9.2 ,  9.  ,  8.7 ,  9.65,  8.8 ,  9.7 , 10.2 ,  9.3 ,  9.  ,
        9.  , 10.1 ,  8.3 ,  8.4 ,  9.2 ,  8.  ,  8.3 ,  8.4 ,  9.9 ,
        8.  ,  8.3 ,  8.9 ,  8.5 ,  9.45,  8.4 ])

In [41]:
p = iqplot.ecdf(bd_1975)
p = iqplot.ecdf(bs_sample, marker_kwargs={'fill_color': None, 'line_color': 'gray'}, p=p)

bokeh.io.show(p)

# See slightly different

In [42]:
n_reps = 100_000

bs_reps_1975 = np.empty(n_reps)

for i in range(n_reps):
    bs_sample = rg.choice(bd_1975, size=len(bd_1975))
    bs_reps_1975[i] = np.mean(bs_sample)

In [50]:
conf_int_1975 = np.percentile(bs_reps_1975, [2.5, 97.5])

In [44]:
def draw_bs_reps(data, func, rg):
    bs_sample = rg.choice(data, size=len(data))
    return func(bs_sample)

In [45]:
bs_reps_2012 = np.array([draw_bs_reps(bd_2012, np.mean, rg) for _ in range(n_reps)])

In [51]:
conf_int_2012 = np.percentile(bs_reps_2012, [2.5, 97.5])

In [52]:
# The confidence interval barelly overlap

In [53]:
years = ['2012', '1975'] # strings because we want categorical access
p = bokeh.plotting.figure(
    frame_height=100,
    frame_width=250,
    x_axis_label='mean beak depth (mm)',
    y_range=years
)

p.circle([np.mean(bd_2012), np.mean(bd_1975)], years, size=5)
p.line(conf_int_1975, ['1975'] * 2, line_width=3)
p.line(conf_int_2012, ['2012'] * 2, line_width=3)

bokeh.io.show(p)

In [54]:
bs_reps_diff_mean = bs_reps_2012 - bs_reps_1975

In [55]:
np.percentile(bs_reps_diff_mean, [2.5, 97.5])

array([0.06301854, 0.39380412])

In [56]:
# 95% of times difference of means favors 2012

In [57]:
np.percentile(bs_reps_diff_mean, [5, 100])

array([0.08891762, 0.5908347 ])

In [59]:
np.sum(bs_reps_diff_mean < 0) / n_reps

0.00371

In [60]:
# less than 1% !

In [61]:
# How to report confidence interval
# Like previous plot. see picture can overlap destribution on it.
# in text:
# 8.96 + 0.11 mm
#      - 0.12 

# Not 8.96 +-0.11 mm (cf. cannot distinguish difference + and -, often do not know how it was computed)

In [64]:
bl_1975 = df.loc[df['year'] == 1975, 'beak length (mm)'].values

In [65]:
bd_1975

array([ 8.4 ,  8.8 ,  8.4 ,  8.  ,  7.9 ,  8.9 ,  8.6 ,  8.5 ,  8.9 ,
        9.1 ,  8.6 ,  9.8 ,  8.2 ,  9.  ,  9.7 ,  8.6 ,  8.2 ,  9.  ,
        8.4 ,  8.6 ,  8.9 ,  9.1 ,  8.3 ,  8.7 ,  9.6 ,  8.5 ,  9.1 ,
        9.  ,  9.2 ,  9.9 ,  8.6 ,  9.2 ,  8.4 ,  8.9 ,  8.5 , 10.4 ,
        9.6 ,  9.1 ,  9.3 ,  9.3 ,  8.8 ,  8.3 ,  8.8 ,  9.1 , 10.1 ,
        8.9 ,  9.2 ,  8.5 , 10.2 , 10.1 ,  9.2 ,  9.7 ,  9.1 ,  8.5 ,
        8.2 ,  9.  ,  9.3 ,  8.  ,  9.1 ,  8.1 ,  8.3 ,  8.7 ,  8.8 ,
        8.6 ,  8.7 ,  8.  ,  8.8 ,  9.  ,  9.1 ,  9.74,  9.1 ,  9.8 ,
       10.4 ,  8.3 ,  9.44,  9.04,  9.  ,  9.05,  9.65,  9.45,  8.65,
        9.45,  9.45,  9.05,  8.75,  9.45,  8.35])

In [66]:
bl_1975

array([13.9 , 14.  , 12.9 , 13.5 , 12.9 , 14.6 , 13.  , 14.2 , 14.  ,
       14.2 , 13.1 , 15.1 , 13.5 , 14.4 , 14.9 , 12.9 , 13.  , 14.9 ,
       14.  , 13.8 , 13.  , 14.75, 13.7 , 13.8 , 14.  , 14.6 , 15.2 ,
       13.5 , 15.1 , 15.  , 12.8 , 14.9 , 15.3 , 13.4 , 14.2 , 15.1 ,
       15.1 , 14.  , 13.6 , 14.  , 14.  , 13.9 , 14.  , 14.9 , 15.6 ,
       13.8 , 14.4 , 12.8 , 14.2 , 13.4 , 14.  , 14.8 , 14.2 , 13.5 ,
       13.4 , 14.6 , 13.5 , 13.7 , 13.9 , 13.1 , 13.4 , 13.8 , 13.6 ,
       14.  , 13.5 , 12.8 , 14.  , 13.4 , 14.9 , 15.54, 14.63, 14.73,
       15.73, 14.83, 15.94, 15.14, 14.23, 14.15, 14.35, 14.95, 13.95,
       14.05, 14.55, 14.05, 14.45, 15.05, 13.25])

In [67]:
# Did it in numpy array, since more efficient & faster that panda dataframe 

In [70]:
p = bokeh.plotting.figure(
    frame_height=250,
    frame_width=250,
    x_axis_label='beak depth (mm)',
    y_axis_label='beak length (mm)'
)

p.circle(bd_1975, bl_1975)

bokeh.io.show(p)

In [71]:
# Can look at correlation between beak length and depth
# probability function of length and depth f(l,d)
# can use plugin principle for the correlation
# See manuscrite note.

In [72]:
np.cov(bd_1975, bl_1975)

array([[0.32103023, 0.26503023],
       [0.26503023, 0.56970612]])

In [73]:
cov = np.cov(bd_1975, bl_1975)
correlation = cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1])

In [74]:
def correlation(x, y):
    cov = np.cov(x, y)
    return cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1])

In [75]:
correlation(bd_1975, bl_1975)

0.6197221344707919

In [77]:
# same procedure to have the confidence interval for correlation.
# Resample pairs of dataset: it is called paired bootstrap.

In [99]:
def draw_bs_pairs(x, y, rg):
    bs_inds = rg.choice(np.arange(len(x)), len(x))
    
    return x[bs_inds], y[bs_inds] # fancy indexing work even when repeats

In [100]:
bs_sample_bl, bs_sample_bd = draw_bs_pairs(bl_1975, bd_1975, rg) # One boostrap samples
bs_corr_rep = correlation(bs_sample_bl, bs_sample_bd) # Give us one replicate
bs_corr_rep

0.7056277498954765

In [101]:
bs_corr_reps = np.empty(n_reps)
for i in range(n_reps):
    bs_sample_bl, bs_sample_bd = draw_bs_pairs(bl_1975, bd_1975, rg) 
    bs_corr_reps[i] = correlation(bs_sample_bl, bs_sample_bd)

In [102]:
np.percentile(bs_corr_reps, [2.5, 97.5])

array([0.4546072 , 0.75210527])

In [110]:
# np_reps_std = np.array([draw_bs_rep(bd_1975, np.std, rg) for _ in range(n_reps)])
# Missed the std code

# Compute replicates
bs_reps_1975 = np.array([draw_bs_reps(bd_1975, np.std, rg) for _ in range(n_reps)])

# Compute confidence interval
conf_int_1975 = np.percentile(bs_reps_1975, [2.5, 97.5])
print(conf_int_1975)

# Plot ECDF
p = iqplot.ecdf(
    bs_reps_1975,
    x_axis_label='std beak depth (mm)',
)

bokeh.io.show(p)

[0.47721603 0.6387646 ]


In [95]:
p = iqplot.ecdf(
    data=df,
    q='beak depth (mm)',
    cats='year',
    conf_int=True, # Gives confidence intervale for ecdf
)

bokeh.io.show(p)

In [96]:
p = iqplot.ecdf(
    data=df,
    q='beak depth (mm)',
    cats='year',
    conf_int=True, # Gives confidence intervale for ecdf
    style = 'staircase'
)

bokeh.io.show(p)

In [ ]:
# dashboarding part of the paper of the future